# Image GANs for Generating Dogs 
We've been learning about AI-generated images in our module on Vision, Images, and Art. Many computer-generated images, like deep fake videos, leverage something called Generative Adversarial Netowrks (GANs). In this homework, we explore using existing GANs libraries for image generation!

Here you will be using a generative adversarial network trained on [ImageNet](http://www.image-net.org/), which we've discussed in lecture previously. Although there are many non-dog classes in ImageNet, we will be focusing our experiments on dog photos. A high quality GAN is tricky to design well and takes dozens of hours or days to train, so to save ourselves computation time, we are using a pre-trained GAN with [TensorFlow](https://www.tensorflow.org/).

# Part 1: Setup (15% effort)
Your goal for this setup part is simply to successfully run all the code below. Give yourself some credit if you manage to get it working!

This assignment requires a little funky module installation. Remember: the course discussion forum and Student Help Hours are your friend if you hit installation errors, don’t suffer alone!

**HIGHLY** recommend you read the Set-up instructions detailed in the GitHub README for this Notebook!

### Environment Setup 

This code uses TensorFlow version **1.15** and Python 3, along with a few other modules.

_Hint_: If you're getting errors on the cell below, you're missing modules! Take notes of which ones you need to install, close this notebook, and view the Github README for set-up instructions!

In [ ]:
from io import BytesIO
import IPython.display
import numpy as np
import urllib
import PIL.Image
from scipy.stats import truncnorm
from skimage import io, data, transform 
import requests
import tensorflow as tf # pip install tensorflow==1.15
from tensorflow.python.framework import ops
import tensorflow_hub as hub #pip install tensorflow_hub
import scipy.misc
from tqdm import tqdm
import random
import cv2 # pip install opencv-python

### BigGAN Model Setup

For this exercise, we will use a pre-trained model called _BigGAN_ generator available on [TensorFlow Hub](https://tfhub.dev/deepmind/biggan-128/2). For more information about this model, check out the authors' paper [_"Large Scale GAN Training for High Fidelity Natural Image Synthesis"_ Brock et al. 2019](https://arxiv.org/abs/1809.11096). Next is the address to download this model from TensorFlow Hub:

In [ ]:
# this model will output 128 by 128 pixel images.
module_path ='https://tfhub.dev/deepmind/biggan-128/2'  
print(module_path)

### Helper Code Setup

This code, in `helpers.py` is adapted from the [BigGANs Tutorial](https://colab.research.google.com/drive/1rqDwIddy0eunhhV8yrznG4SNiB5XWFJJ) from [Machine Learning for Artists](https://ml4a.github.io/) by Gene Kogan. Many of the exercises here are inspired from that tutorial, so check it out if you want to have more fun with GANs later!

In [ ]:
from helpers import GANSession
# you can safely ignore the tensorflow WARNING outputs
gan = GANSession(module_path)
print("This cell takes Iris' laptop 25 minutes to run (Colab, 5 mins).")

## Part 2: Experimenting with generating deep fake puppies

### Task 2.A (2% effort) Choose a dog breed from `dog_classes.txt`, replacing the "263...'corgi" below.

In [ ]:
## Task 2.A: Modify this code!
truncation = random.uniform(0.02,1) # min:0.02, max:1
noise_seed = random.randint(0,100) # min:0, max:100
category = "263: 'Pembroke, Pembroke Welsh corgi'" # put dog breed here

num_samples = 1 # Number of images to generate with these parameters

# returns an ndarray, each item is another ndarray is used to generate the images
z = gan.truncated_z_sample(num_samples, truncation, noise_seed) 
y = int(category.split(':')[0])

print('truncation: ',truncation, 'noise seed:',noise_seed)

ims = gan.sample(z, y, truncation=truncation) # ims is a numpy array
max_columns = 20
gan.imshow(gan.imgrid(ims, cols=min(num_samples, max_columns)))

### Task 2.B (8% effort) Initial Explorations
Run the above image generator cell a few times. Below, paste in your best looking result as well as the `truncation` and `noise seed` parameter values for that result (outputted by the program). Next paste in your worst looking dog as well as its parameter values. Describe what you are seeing: what kinds of visual errors do you think this model is creating? 

_Hint:_ if you copy a screenshot to clipboard, you can paste it directly into this markdown cell.

_Double click this text to write your answer to the question here._

### Task 2.C (8% effort) Generate a row of dogs with different noise_seeds
The code below generates two images with the same truncation value, but with two different noise_seeds. Adapt this code so that it generates a row of 11 dog images, where each dog has a different value of the parameter `noise_seed` evenly distributed across the range 0 to 100 inclusive (i.e., 0, 10, 20, ...100).

_For a Welsh Pembroke Corgi with a fixed `truncation` value of `0.95` this looks like:_
![image.png](attachment:image.png)

_where the leftmost Corgi has a `noise_seed` of `0` and the rightmost has a `noise_seed` of 100_

In [ ]:
## Task 2.C: Modify this code!
truncation = 0.95 # min:0.02, max:1
category = "263: 'Pembroke, Pembroke Welsh corgi'" # put dog breed here

num_samples = 1
z_list = []

# Grab the first element returned by truncated_z_sample
z_item = gan.truncated_z_sample(num_samples, truncation, 0)
# Add that first element to our list of np.ndarray
z_list.append(z_item[0])
# Repeat
z_item = gan.truncated_z_sample(num_samples, truncation, 100)
z_list.append(z_item[0])

z = np.asarray(z_list) # convert our list into an np.ndarray
y = int(category.split(':')[0])

ims = gan.sample(z, y, truncation=truncation)
gan.imshow(gan.imgrid(ims,cols=11))

### Task 2.D (12% effort) Generate a grid of dogs, varying noise_seed and truncation
Take the code from 2.C above and copy that below. Now, your target output is a _grid_ of dogs. Just as before in 2.C, each _column_ will be 11 generated dogs in an ordered range by `noise_seed` 0-100. Now the _rows_ of the grid shall be different values of the `truncation` parameter, in an ordered range from 0.02-1.0. Your finished grid should have 11 rows, 11 columns, where the top left dog in the grid has a `noise_seed = 0` and `truncation = 0.02` and the values of those parameters increase top to bottom left to right. The dog at the bottom right will have a `noise_seed = 100` and `truncation = 1.0`.

In [ ]:
## Task 2.D: Add new code here!

### Task 2.E (15% effort) Hypothesize the relationship between noise_seed and truncation. 
Given your experimentations above and your grid of dogs, hypothesize:
 - What might be the relationship between `noise_seed` and what a generated dog looks like?
 - What might be the relationship between `truncation` and what a generated dog looks like?

_Double click this text to write your answer to the question here._

# Part 3: Experimenting with transforming puppies

In this section, we will experiment with different forms of interpolation to transform and combine different dog images.

### Task 3.A (10% effort) Compare truncation and noise_seed across breeds
Modify and run the code below, putting in your favorite `truncation` and `noise_seed` values from your experiments in Part 2. Run the code with a pair of closely similar breeds (similar to the Pembroke and the Cardigan Welsh corgies shown below) and a pair of very different breeds. 
 - What do you hypothesize this code is doing with the two dog image samples?
 - Hypothesize: Does the performance (visual quality of the output) differ based on qualities of the source images such as similar dog breed or similar image background?
 
_Double click this text to write your answer to the question here._

In [ ]:
## Task 3.A: Modify this code!
num_interps = 10 #min:2, max:1000
truncation = 0.46 #min:0.02, max:1
noise_seed_A = 2 #min:0, max:100
category_A = "263: 'Pembroke, Pembroke Welsh corgi'" 
noise_seed_B = 99 #min:0, max:100
category_B = "264: 'Cardigan, Cardigan Welsh corgi'" 

y_interp, z_interp = gan.get_interpolated_yz([int(category_A.split(':')[0]), int(category_B.split(':')[0])], num_interps, noise_seed_A, noise_seed_B, truncation=truncation)
imgs = gan.sample(z_interp, y_interp, truncation=truncation)
gan.imshow(gan.imgrid(imgs, cols=num_interps))

### Task 3.B (8% effort) Different truncations for different dalmations
The code below does a different, but similar operation to the code in Task 3.A.

Modify and run the code below, putting in your favorite `truncation` and `noise_seed` values from your experiments in Part 2. Run the code with a pair of closely similar breeds (similar to the Pembroke and the Cardigan Welsh corgies shown below) and a pair of very different breeds. 
 - What do you hypothesize this code is doing with the two dog image samples?
 - Hypothesize: Does the performance (visual quality of the output) differ based on qualities of the source images such as similar dog breed or similar image background?
 - Unlike in Task 3.A, both images share the same `noise_seed`. From your observations, what do you think is the effect of having the images share the same parameter value versus having each their own `noise_seed`?
 
_Double click this text to write your answer to the question here._

In [ ]:
## Task 3.B: Modify this code!
truncation = 0.45 #min:0.02, max:1
noise_seed = 22 #min:0, max:100
categoryA = "263: 'Pembroke, Pembroke Welsh corgi'" 
categoryB = "264: 'Cardigan, Cardigan Welsh corgi'" 

categories = [int(categoryA.split(':')[0]), int(categoryB.split(':')[0])]
y, z = gan.get_combination_yz(categories, noise_seed, truncation)
imgs = gan.sample(z, y, truncation=truncation)
gan.imshow(gan.imgrid(imgs, cols=len(categories)+1))

### Task 3.C (5% effort) Paste in below your best looking result from 3.B. Include a description of all parameters you used to generate the image.

_Double click this text to write your answer to the question here._

### Task 3.D (5% effort) reallyBigGANs
A big advantage of the BigGAN model over other GANs is that it is able to produce much higher resolution images than GANs were previously capable of. To test this out, go up to the BigGAN Model Setup section, and replace the module path with ['https://tfhub.dev/deepmind/biggan-512/2']('https://tfhub.dev/deepmind/biggan-512/2'). Then re-run just the **Setup** section of this notebook. This will load a version of BigGAN that can generate 512 by 512 pixel images instead of 128 by 128 pixels.

Now re-run the code for Task 3.B, and paste in below your best looking result, including a description of all parameters. _note: since this generates a larger image, it will take longer to compute (takes Iris' laptop about 1 minute)_

_Double click this text to write your answer to the question here._

### Task 3.E (8% effort) BigGAN vs not-so-bigGAN
Compare your best results from Task 3.C and 3.D. Hypothesize, what do you think the effect of a higher resolution GAN is on the image quality?

_Double click this text to write your answer to the question here._

# Assignment Submission

Once you've completed all of the above, you're done with assignment 5! As always, clean up your code and ensure your entire Jupyter Notebook runs before submitting, Iris must be able to run your notebook on her machine.

Once you think everything is set, please change the filename of your notebook to `[yourunixID]_haii20[assignmentnumber].ipynb`, e.g., `ikh1_haii20a5.ipynb` and then submit your notebook on Glow.